In [2]:
!pip install boto3
!pip install opencv-python
!pip install scikit-image

  Using cached opencv_python-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (63.0 MB)


In [3]:
!pip install boto3 numpy Pillow scikit-image tqdm
!pip install opencv-python

#Code version1

In [ ]:
import sys
import subprocess
import os
import json
import time
import logging
import boto3
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops
from skimage import img_as_ubyte
from tqdm import tqdm

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# AWS S3 bucket and dataset path
BUCKET_NAME = 'sagemaker-studio-010526272250-hsf94lgtf6i'
S3_DATA_PATH = 'Dentex_Challenge_Project_Image_Dataset_ultrareduced/'
LOCAL_DATA_PATH = 'local_images'
OUTPUT_PATH = 'segmented_images'
METADATA_FILE = 'metadata.json'  # File to store segmentation metadata

# Tooth mapping (placeholder)
TOOTH_MAPPING = {
    "image1.png": {"quadrant": "upper_right", "tooth": "18"},
    "image2.png": {"quadrant": "upper_left", "tooth": "28"},
    "image3.png": {"quadrant": "lower_left", "tooth": "38"},
    "image4.png": {"quadrant": "lower_right", "tooth": "48"}
}

# GLCM Parameters
DISTANCE = [5]
ANGLE = [0, np.pi/4, np.pi/2, 3*np.pi/4]
LEVELS = 256
WINDOW_SIZE = 5 # Define window size here

def install_dependencies():
    """Install required system and Python packages."""
    try:
        # Update package list
        try:
            subprocess.check_call(['sudo', 'apt-get', 'update'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            logging.info("apt-get update successful.")
        except subprocess.CalledProcessError as e:
            logging.error(f"Error updating apt-get: {e}")
            logging.error("Please ensure you have sudo privileges or the appropriate permissions to run apt-get update.")
            sys.exit(1)

        # Install system dependencies
        system_packages = [
            'libgl1-mesa-glx',
            'libglib2.0-0',
            'libsm6',
            'libxext6',
            'libxrender-dev'
        ]
        logging.info("Installing system packages...")
        for package in system_packages:
            try:
                subprocess.check_call(['sudo', 'apt-get', 'install', '-y', package], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
                logging.info(f"Successfully installed system package: {package}")
            except subprocess.CalledProcessError as e:
                logging.error(f"Error installing system package {package}: {e}")
                logging.error("Please ensure you have sudo privileges or the appropriate permissions to install system packages.")
                sys.exit(1)

        # Install Python packages
        python_packages = [
            'opencv-python-headless',
            'numpy',
            'boto3',
            'scikit-image',
            'tqdm'
        ]
        logging.info("Installing python packages...")
        for package in python_packages:
            try:
                subprocess.check_call([sys.executable, '-m', 'pip', 'install', package], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
                logging.info(f"Successfully installed python package: {package}")
            except subprocess.CalledProcessError as e:
                logging.error(f"Error installing python package {package}: {e}")
                sys.exit(1)

    except subprocess.CalledProcessError as e:
        logging.error(f"Error installing dependencies: {e}")
        sys.exit(1)
    except Exception as e:
        logging.error(f"Unexpected error during installation: {e}")
        sys.exit(1)

def download_images_from_s3(bucket_name, s3_path, local_path):
    """Downloads images from S3 to a local directory."""
    try:
        s3 = boto3.client('s3')

        if not os.path.exists(local_path):
            os.makedirs(local_path)

        objects = s3.list_objects_v2(Bucket=bucket_name, Prefix=s3_path)
        if 'Contents' not in objects:
            logging.warning(f"No files found in S3 path: {s3_path}")
            return []

        files_to_download = [obj['Key'] for obj in objects.get('Contents', [])
                           if obj['Key'].lower().endswith(('.png', '.jpg', '.jpeg'))]

        for s3_file_path in tqdm(files_to_download, desc="Downloading images from S3"):
            local_file_path = os.path.join(local_path, os.path.basename(s3_file_path))
            try:
                s3.download_file(bucket_name, s3_file_path, local_file_path)
                logging.info(f"Downloaded {s3_file_path} to {local_file_path}")
            except Exception as e:
                logging.error(f"Error downloading {s3_file_path}: {e}")
        return files_to_download # Return the list of files

    except boto3.exceptions.BotoClientError as e:
        logging.error(f"AWS S3 error: {e}")
        sys.exit(1)
    except Exception as e:
        logging.error(f"Error in download_images_from_s3: {e}")
        sys.exit(1)

def calculate_glcm_features(image):
    """Calculates GLCM features."""
    try:
        gray_image = img_as_ubyte(image)
        glcm = graycomatrix(gray_image, distances=DISTANCE, angles=ANGLE,
                           levels=LEVELS, symmetric=True, normed=True)

        if glcm.size == 0: # Check if glcm is empty
            #logging.warning("GLCM is empty. Returning None.")  #  Avoid excessive logging
            return None

        contrast = graycoprops(glcm, 'contrast')[0, 0]
        correlation = graycoprops(glcm, 'correlation')[0, 0]
        energy = graycoprops(glcm, 'energy')[0, 0]
        homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]

        return (contrast, correlation, energy, homogeneity)
    except Exception as e:
        logging.error(f"Error in calculate_glcm_features: {e}")
        return None


def segment_image(image, contrast_threshold, correlation_threshold,
                 energy_threshold, homogeneity_threshold):
    """Segments the image based on GLCM feature thresholds using a sliding window approach."""
    try:
        # Precompute GLCM features for all windows
        half_window = WINDOW_SIZE // 2
        glcm_features = np.zeros((image.shape[0] - 2 * half_window, image.shape[1] - 2 * half_window, 4))  # (height, width, features)
        mask = np.zeros(image.shape[:2], dtype=np.uint8)

        # Determine total number of steps for progress tracking
        total_steps = (image.shape[0] - 2 * half_window) * (image.shape[1] - 2 * half_window)

        # Initialize tqdm for progress tracking
        with tqdm(total=total_steps, desc="Segmenting Image", unit="pixel") as pbar:
            # Loop through all possible window positions
            for i in range(half_window, image.shape[0] - half_window):
                for j in range(half_window, image.shape[1] - half_window):
                    # Extract window
                    window = image[i - half_window:i + half_window + 1, j - half_window:j + half_window + 1]

                    # Ensure window is the correct size
                    if window.shape != (WINDOW_SIZE, WINDOW_SIZE):
                        continue

                    # Calculate GLCM features
                    glcm_features_val = calculate_glcm_features(window)

                    if glcm_features_val is not None:
                        contrast, correlation, energy, homogeneity = glcm_features_val
                        # Apply thresholds
                        if (contrast > contrast_threshold and
                            correlation < correlation_threshold and
                            energy > energy_threshold and
                            homogeneity < homogeneity_threshold):
                            mask[i, j] = 255
                    else:
                        mask[i, j] = 0 # set the value to 0

                    # Update progress bar
                    pbar.update(1)

        return mask
    except Exception as e:
        logging.error(f"Error in segment_image: {e}")
        return None


def process_images(local_path, output_path, tooth_mapping, image_files):
    """Processes images, segments them, and saves the segmented images and metadata."""
    metadata = {}  # Store metadata for each image

    try:
        if not os.path.exists(output_path):
            os.makedirs(output_path)

        # image_files = [f for f in os.listdir(local_path)  # This is now passed as an argument
        #              if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        for filename in tqdm(image_files, desc="Processing images"):
            image_path = os.path.join(local_path, filename)

            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if image is None:
                logging.warning(f"Could not read image: {filename}")
                continue

            # Thresholds (adjust as needed)
            contrast_threshold = 0.2
            correlation_threshold = 0.8
            energy_threshold = 0.6
            homogeneity_threshold = 0.5

            segmented_mask = segment_image(image, contrast_threshold,
                                         correlation_threshold,
                                         energy_threshold,
                                         homogeneity_threshold)

            if segmented_mask is None:
                logging.warning(f"Segmentation failed for {filename}")
                continue

            segmented_tooth = cv2.bitwise_and(image, image, mask=segmented_mask)

            # Store Segmentation Metadata
            metadata[filename] = {
                "contrast_threshold": contrast_threshold,
                "correlation_threshold": correlation_threshold,
                "energy_threshold": energy_threshold,
                "homogeneity_threshold": homogeneity_threshold,
                "segmentation_points": np.argwhere(segmented_mask > 0).tolist() if np.any(segmented_mask) else []  # Store coordinates
            }

            # Save the Segmented Image
            output_filename = f"unlabeled_{filename}"
            quadrant = "unknown"
            tooth_level = "unknown"

            if filename in tooth_mapping:
                quadrant = tooth_mapping[filename]["quadrant"]
                tooth_level = tooth_mapping[filename]["tooth"]
                metadata[filename]["quadrant"] = quadrant
                metadata[filename]["tooth_level"] = tooth_level

            output_filepath = os.path.join(output_path, output_filename)
            cv2.imwrite(output_filepath, segmented_tooth)
            logging.info(f"Processed and saved segmented image to {output_filepath}")

    except Exception as e:
        logging.error(f"Error in process_images: {e}")
        sys.exit(1)

    # Save Metadata to JSON
    try:
        with open(os.path.join(output_path, METADATA_FILE), 'w') as f:
            json.dump(metadata, f, indent=4)
        logging.info(f"Saved metadata to {os.path.join(output_path, METADATA_FILE)}")
    except Exception as e:
        logging.error(f"Error saving metadata: {e}")

# First, ensure all dependencies are installed
if __name__ == "__main__":
    install_dependencies()

# Now import the required packages
try:
    import boto3
    import cv2
    import numpy as np
    from skimage.feature import graycomatrix, graycoprops
    from skimage import img_as_ubyte
    from skimage.filters import threshold_otsu
    from tqdm import tqdm
except ImportError as e:
    logging.error(f"Error importing packages after installation: {e}")
    logging.error("Please try running the script again.")
    sys.exit(1)

if __name__ == "__main__":
    start_time = time.time()
    try:
        # Download images from S3
        image_files = download_images_from_s3(BUCKET_NAME, S3_DATA_PATH, LOCAL_DATA_PATH)  # Capture the list of downloaded files

        # Process and segment images
        if image_files: #Only call process_images if there were files downloaded
            process_images(LOCAL_DATA_PATH, OUTPUT_PATH, TOOTH_MAPPING, image_files) #Pass the list of images to be processed

        logging.info("Image segmentation completed successfully.")
    except Exception as e:
        logging.error(f"Main process error: {e}")
        sys.exit(1)
    finally:
        end_time = time.time()
        logging.info(f"Total execution time: {end_time - start_time:.2f} seconds")

#Code Version3

In [ ]:
import boto3
import os
import numpy as np

# AWS S3 Configuration
S3_BUCKET = 'sagemaker-studio-010526272250-hsf94lgtf6i'  # Replace with your bucket name
S3_IMAGE_PATH = 'Dentex_Challenge_Project_Image_Dataset_ultrareduced/' # Path to your panoramic X-ray images
LOCAL_OUTPUT_DIR = 'segmented_teeth'  # Directory to save segmented images locally

# Ensure the local output directory exists
os.makedirs(LOCAL_OUTPUT_DIR, exist_ok=True)

# Initialize S3 client
s3 = boto3.client('s3')

def install_opencv():
    """Installs OpenCV and its dependencies with sudo."""
    try:
        print("Attempting to install OpenCV and its dependencies...")
        os.system("sudo apt-get update")
        os.system("sudo apt-get install -y libgl1-mesa-glx")
        os.system("sudo pip install opencv-python")  # Install opencv-python after installing libgl
        print("OpenCV and its dependencies installed successfully.")
        return True
    except Exception as e:
        print(f"Error installing OpenCV and its dependencies: {e}")
        print("Please ensure you have the necessary permissions and that apt-get is available.")
        return False

# Try importing cv2, and install if it fails
try:
    import cv2
    print("OpenCV is already installed.")
except ImportError:
    print("OpenCV is not installed. Attempting to install...")
    if install_opencv():
        import cv2  # Try importing again after installation
        print("OpenCV installed successfully.")
    else:
        print("OpenCV installation failed. Exiting...")
        exit()

def download_and_segment_teeth(image_name):
    """
    Downloads a panoramic X-ray image from S3, performs teeth segmentation,
    and saves the segmented teeth images locally.

    Args:
        image_name (str): The name of the image file in S3.
    """

    try:
        # 1. Download the image from S3
        s3_image_key = os.path.join(S3_IMAGE_PATH, image_name)
        local_image_path = os.path.join(LOCAL_OUTPUT_DIR, image_name)

        print(f"Downloading {image_name} from S3...")
        s3.download_file(S3_BUCKET, s3_image_key, local_image_path)
        print(f"Downloaded {image_name} to {local_image_path}")

        # 2. Load the image using OpenCV
        img = cv2.imread(local_image_path, cv2.IMREAD_GRAYSCALE) # Load as grayscale for segmentation
        if img is None:
            print(f"Error: Could not load image {image_name}")
            return

        # 3. Image Segmentation (Placeholder - Replace with your actual segmentation logic)
        # This is the most critical part.  You MUST replace this with your actual segmentation method.
        # The example here is a very basic thresholding, which is unlikely to be sufficient
        # for real panoramic X-ray images.

        print(f"Segmenting teeth in {image_name}...")

        #Example segmentation, very basic and likely ineffective:
        thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)[1]  # Simple thresholding

        # Find contours
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Filter contours (remove small noise)
        min_contour_area = 50  # Adjust this value based on your image resolution and teeth size
        filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]

        # Create a directory for each image to store segmented teeth
        image_output_dir = os.path.join(LOCAL_OUTPUT_DIR, os.path.splitext(image_name)[0])
        os.makedirs(image_output_dir, exist_ok=True)


        # Iterate through contours and crop the teeth
        for i, contour in enumerate(filtered_contours):
            x, y, w, h = cv2.boundingRect(contour)
            cropped_tooth = img[y:y + h, x:x + w]

            # Save each segmented tooth
            tooth_filename = os.path.join(image_output_dir, f"tooth_{i}.png")
            cv2.imwrite(tooth_filename, cropped_tooth)
            print(f"Saved tooth {i} to {tooth_filename}")


        print(f"Segmented teeth and saved to {image_output_dir}")


    except Exception as e:
        print(f"Error processing image {image_name}: {e}")



def main():
    """
    Lists images in the specified S3 path and processes each one.
    """
    try:
        # 1. List all images in the S3 path
        print(f"Listing images in s3://{S3_BUCKET}/{S3_IMAGE_PATH}")
        response = s3.list_objects_v2(Bucket=S3_BUCKET, Prefix=S3_IMAGE_PATH)

        # 2. Extract image names from the response
        image_names = [obj['Key'].split('/')[-1] for obj in response.get('Contents', []) if obj['Key'].endswith(('.png', '.jpg', '.jpeg'))]  # Filter for image files
        print(f"Found {len(image_names)} images.")

        # 3. Process each image
        for image_name in image_names:
            download_and_segment_teeth(image_name)

    except Exception as e:
        print(f"An error occurred: {e}")


if __name__ == "__main__":
    main()

#Code Version4

In [ ]:
import boto3
import os
import cv2
import numpy as np
from io import BytesIO
import logging  # Import the logging module

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# AWS S3 Configuration
S3_BUCKET = 'sagemaker-studio-010526272250-hsf94lgtf6i'  # Replace with your bucket name
S3_IMAGE_PATH = 'Dentex_Challenge_Project_Image_Dataset_ultrareduced/'  # Path to your panoramic X-ray images
S3_SEGMENTED_PATH = 'segmented_teeth_output/'  # Path to save segmented teeth in S3

# Local Configuration
LOCAL_IMAGE_DIR = 'temp_images'  # Local directory to temporarily store images
os.makedirs(LOCAL_IMAGE_DIR, exist_ok=True)

# Initialize S3 client
s3 = boto3.client('s3')


def install_opencv():
    """Installs OpenCV and its dependencies with sudo."""
    try:
        logging.info("Attempting to install OpenCV and its dependencies...")
        os.system("sudo apt-get update")
        os.system("sudo apt-get install -y libgl1-mesa-glx")
        os.system("sudo pip install opencv-python")  # Install opencv-python after installing libgl
        logging.info("OpenCV and its dependencies installed successfully.")
        return True
    except Exception as e:
        logging.error(f"Error installing OpenCV and its dependencies: {e}")
        logging.error("Please ensure you have the necessary permissions and that apt-get is available.")
        return False


# Try importing cv2, and install if it fails
try:
    import cv2

    logging.info("OpenCV is already installed.")
except ImportError:
    logging.warning("OpenCV is not installed. Attempting to install...")
    if install_opencv():
        import cv2  # Try importing again after installation

        logging.info("OpenCV installed successfully.")
    else:
        logging.error("OpenCV installation failed. Exiting...")
        exit()


def download_and_segment_teeth(image_name):
    """
    Downloads a panoramic X-ray image from S3, performs teeth segmentation,
    and saves the segmented teeth images back to the S3 bucket.

    Args:
        image_name (str): The name of the image file in S3.
    """

    try:
        # 1. Download the image from S3
        s3_image_key = os.path.join(S3_IMAGE_PATH, image_name)
        local_image_path = os.path.join(LOCAL_IMAGE_DIR, image_name)  # Temporary local path

        logging.info(f"Downloading {image_name} from S3...")
        s3.download_file(S3_BUCKET, s3_image_key, local_image_path)
        logging.info(f"Downloaded {image_name} to {local_image_path}")

        # 2. Load the image using OpenCV
        img = cv2.imread(local_image_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale for segmentation
        if img is None:
            logging.error(f"Error: Could not load image {image_name}")
            return

        # 3. Image Segmentation (Improved)
        logging.info(f"Segmenting teeth in {image_name}...")

        # Denoising
        img = cv2.GaussianBlur(img, (5, 5), 0)

        # Adaptive Thresholding (Crucial for varying lighting)
        thresh = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

        # Morphological Operations to close gaps and remove noise
        kernel = np.ones((3, 3), np.uint8)
        closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
        opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel, iterations=2)

        # Find contours
        contours, _ = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Filter contours (remove small noise and non-tooth shapes using aspect ratio)
        min_contour_area = 500  # Increased minimum area for better filtering.  Adjust as needed.
        max_contour_area = 50000 #Add max contour area to remove big noises
        min_aspect_ratio = 0.2 #min aspect ratio
        max_aspect_ratio = 5

        filtered_contours = []
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if min_contour_area < area < max_contour_area:
                x, y, w, h = cv2.boundingRect(cnt)
                aspect_ratio = float(w) / h
                if min_aspect_ratio < aspect_ratio < max_aspect_ratio: #Check aspect ratio to remove unwanted contours
                    filtered_contours.append(cnt)

        # Iterate through contours and crop the teeth
        for i, contour in enumerate(filtered_contours):
            x, y, w, h = cv2.boundingRect(contour)
            cropped_tooth = img[y:y + h, x:x + w]

            # 4. Save each segmented tooth directly to S3
            # Create a unique key for the segmented tooth in S3
            s3_tooth_key = os.path.join(S3_SEGMENTED_PATH, f"{os.path.splitext(image_name)[0]}_tooth_{i}.png")
            logging.info(f"Saving tooth {i} to s3://{S3_BUCKET}/{s3_tooth_key}")

            # Convert the cropped tooth image to bytes (PNG format) using BytesIO
            is_success, im_buf_arr = cv2.imencode(".png", cropped_tooth)  # encode as png
            if is_success:
                img_bytes = im_buf_arr.tobytes()

                # Upload the bytes to S3
                try:
                    s3.upload_fileobj(BytesIO(img_bytes), S3_BUCKET, s3_tooth_key, ExtraArgs={'ContentType': 'image/png'})
                    logging.info(f"Tooth {i} saved successfully to S3.")

                except Exception as e:
                    logging.error(f"Error uploading tooth {i} to S3: {e}")
            else:
                logging.error(f"Error encoding tooth {i} as PNG.")

        logging.info(f"Segmented teeth and saved to s3://{S3_BUCKET}/{S3_SEGMENTED_PATH}")
        # Cleanup local image file:
        os.remove(local_image_path)

    except Exception as e:
        logging.error(f"Error processing image {image_name}: {e}")


def main():
    """
    Lists images in the specified S3 path and processes each one.
    """
    try:
        # 1. List all images in the S3 path
        logging.info(f"Listing images in s3://{S3_BUCKET}/{S3_IMAGE_PATH}")
        response = s3.list_objects_v2(Bucket=S3_BUCKET, Prefix=S3_IMAGE_PATH)

        # 2. Extract image names from the response
        image_names = [obj['Key'].split('/')[-1] for obj in response.get('Contents', []) if
                       obj['Key'].endswith(('.png', '.jpg', '.jpeg'))]  # Filter for image files
        logging.info(f"Found {len(image_names)} images.")

        # 3. Process each image
        for image_name in image_names:
            download_and_segment_teeth(image_name)

    except Exception as e:
        logging.error(f"An error occurred: {e}")


# Delete the temporary image dir after completion.
def cleanup_local_dir():
    try:
        os.rmdir(LOCAL_IMAGE_DIR)
        logging.info(f"Successfully removed temporary directory: {LOCAL_IMAGE_DIR}")
    except OSError as e:
        logging.warning(f"Could not remove temporary image directory at {LOCAL_IMAGE_DIR}, it may not be empty. Please remove manually. Error: {e}")


if __name__ == "__main__":
    main()
    cleanup_local_dir()

In [ ]:
    import boto3
    import io
    from PIL import Image
    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg
    import os
    
    # AWS S3 Configuration
    S3_BUCKET = 'sagemaker-studio-010526272250-hsf94lgtf6i'  # Replace with your bucket name
    S3_SEGMENTED_PATH = 'segmented_teeth_output/'  # Path to segmented teeth in S3
    LOCAL_IMAGE_DIR = 'downloaded_images'
    os.makedirs(LOCAL_IMAGE_DIR, exist_ok=True)
    
    
    # Initialize S3 client
    s3 = boto3.client('s3')
    
    
    def list_images_in_s3(bucket, prefix):
        """Lists all image files in the specified S3 bucket and prefix."""
        image_keys = []
        try:
            response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
            if 'Contents' in response:
                for obj in response['Contents']:
                    if obj['Key'].lower().endswith(('.png', '.jpg', '.jpeg')):
                        image_keys.append(obj['Key'])
            else:
                print(f"No objects found in s3://{bucket}/{prefix}")
        except Exception as e:
            print(f"Error listing objects in S3: {e}")
        return image_keys
    
    
    def download_image_from_s3(bucket, key, local_path):
        """Downloads an image from S3 to a local file."""
        try:
            s3.download_file(bucket, key, local_path)
            print(f"Downloaded s3://{bucket}/{key} to {local_path}")
            return True
        except Exception as e:
            print(f"Error downloading {key} from S3: {e}")
            return False
    
    def display_images(image_paths, images_per_row=5):
        """Displays images in a grid using Matplotlib."""
        num_images = len(image_paths)
        if num_images == 0:
            print("No images to display.")
            return
    
        num_rows = (num_images + images_per_row - 1) // images_per_row  # Calculate number of rows
        fig, axes = plt.subplots(num_rows, images_per_row, figsize=(15, 3 * num_rows)) #Adjust fig size as needed
    
        # Flatten axes array for easier indexing
        if num_rows == 1:
            axes = [axes]  # Handle the case of only one row
        axes = [ax for sublist in axes for ax in sublist]
    
        for i, image_path in enumerate(image_paths):
            try:
                img = mpimg.imread(image_path)
                axes[i].imshow(img)
                axes[i].axis('off')  # Hide axes
                axes[i].set_title(os.path.basename(image_path)) # add image name to title
            except FileNotFoundError:
                print(f"File not found: {image_path}")
            except Exception as e:
                print(f"Error displaying image {image_path}: {e}")
    
        # Hide any unused subplots
        for i in range(num_images, len(axes)):
            axes[i].axis('off')
    
        plt.tight_layout()
        plt.show()
    
    
    def main():
        """Lists images in S3, downloads them, and displays them."""
        image_keys = list_images_in_s3(S3_BUCKET, S3_SEGMENTED_PATH)
    
        if not image_keys:
            print("No images found in the specified S3 path.")
            return
    
        # Download images to a local directory
        local_image_paths = []
        for key in image_keys:
            local_image_path = os.path.join(LOCAL_IMAGE_DIR, os.path.basename(key))
            if download_image_from_s3(S3_BUCKET, key, local_image_path):
                local_image_paths.append(local_image_path)
    
        # Display the downloaded images
        if local_image_paths:
            display_images(local_image_paths)
    
        # Cleanup local directory (optional):
        # Warning: This will delete the downloaded images.
        # Uncomment if you want to delete the images after viewing.
        # import shutil
        # shutil.rmtree(LOCAL_IMAGE_DIR)
    
    
    if __name__ == "__main__":
        main()

#Code Version5

In [ ]:
import boto3
import io
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import numpy as np
from skimage.feature import greycomatrix, graycoprops
from skimage import img_as_ubyte, img_as_float
from skimage.filters import threshold_otsu, rank, gaussian, sobel
from skimage.measure import label, regionprops
from skimage.morphology import closing, square, remove_small_objects, disk, opening, erosion, dilation
from skimage.color import rgb2gray
from skimage.segmentation import watershed, clear_border
from skimage.exposure import equalize_adapthist
from scipy import ndimage as ndi
from scipy.signal import find_peaks
from tqdm import tqdm
import time
from datetime import datetime

# AWS S3 Configuration
S3_BUCKET = 'sagemaker-studio-010526272250-hsf94lgtf6i'
S3_ORIGINAL_IMAGES_PATH = 'Dentex_Challenge_Project_Image_Dataset_ultrareduced/'
S3_SEGMENTED_PATH = 'segmented_teeth_output/'
LOCAL_IMAGE_DIR = 'downloaded_images'
LOCAL_SEGMENTED_DIR = 'segmented_teeth'
os.makedirs(LOCAL_IMAGE_DIR, exist_ok=True)
os.makedirs(LOCAL_SEGMENTED_DIR, exist_ok=True)

# Enhanced constants
MIN_TOOTH_AREA = 300
MAX_TOOTH_AREA = 15000
ECCENTRICITY_THRESHOLD = 0.85
SOLIDITY_THRESHOLD = 0.45
SMOOTHING_RADIUS = 10
CLAHE_KERNEL = 8
WATERSHED_FOOTPRINT = 3
MIN_DISTANCE_BETWEEN_TEETH = 20

# Dental quadrant mapping
QUADRANT_MAPPING = {
    'upper_right': 1,
    'upper_left': 2,
    'lower_left': 3,
    'lower_right': 4
}

class ProgressTracker:
    def __init__(self):
        self.start_time = time.time()
        self.logs = []
        self.current_image = None
        self.total_images = 0
        self.processed_images = 0
        
    def log(self, message, stage=None):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_entry = f"[{timestamp}] {message}"
        print(log_entry)
        self.logs.append(log_entry)
        
    def update_progress(self, stage, progress):
        self.log(f"Image {self.current_image}: {stage} - {progress}%")
        
    def save_logs(self, filename="segmentation_logs.txt"):
        with open(filename, 'w') as f:
            f.write("\n".join(self.logs))
            
    def get_elapsed_time(self):
        return time.time() - self.start_time

# Initialize global progress tracker
progress = ProgressTracker()

def determine_quadrant_and_position(centroid, image_shape):
    """Determine the dental quadrant and relative position of a tooth."""
    height, width = image_shape
    y, x = centroid
    
    # Determine if upper or lower
    is_upper = y < height / 2
    
    # Determine if left or right
    is_right = x < width / 2
    
    if is_upper and is_right:
        return 1  # Upper right
    elif is_upper and not is_right:
        return 2  # Upper left
    elif not is_upper and not is_right:
        return 3  # Lower left
    else:
        return 4  # Lower right

def save_individual_tooth(image, mask, base_filename, tooth_number, quadrant, output_dir):
    """Save individual tooth image with appropriate naming convention."""
    # Extract the first part of the filename (before underscore)
    base_name = base_filename.split('_')[0]
    
    # Create output filename
    output_filename = f"{base_name}_Q{quadrant}_T{tooth_number}.png"
    output_path = os.path.join(output_dir, output_filename)
    
    # Apply mask to original image
    tooth_image = image.copy()
    tooth_image[~mask] = 0
    
    # Convert to PIL Image and save
    tooth_image_pil = Image.fromarray(img_as_ubyte(tooth_image))
    tooth_image_pil.save(output_path)
    
    return output_path

def enhance_image(img):
    """Apply advanced image enhancement techniques with progress tracking."""
    progress.update_progress("Enhancement", 0)
    
    img_float = img_as_float(img)
    progress.update_progress("Enhancement", 25)
    
    img_clahe = equalize_adapthist(img_float, kernel_size=CLAHE_KERNEL)
    progress.update_progress("Enhancement", 50)
    
    img_smooth = gaussian(img_clahe, sigma=1, preserve_range=True)
    progress.update_progress("Enhancement", 75)
    
    result = img_as_ubyte(img_smooth)
    progress.update_progress("Enhancement", 100)
    
    return result

def detect_teeth_boundaries(img):
    """Detect teeth boundaries with progress tracking."""
    progress.update_progress("Boundary Detection", 0)
    
    gradient = sobel(img)
    progress.update_progress("Boundary Detection", 33)
    
    gradient = gaussian(gradient, sigma=1)
    progress.update_progress("Boundary Detection", 66)
    
    grad_thresh = threshold_otsu(gradient)
    edges = gradient > grad_thresh
    edges = closing(edges, disk(2))
    edges = remove_small_objects(edges, min_size=50)
    
    progress.update_progress("Boundary Detection", 100)
    return edges

def find_peaks_2d(img):
    """Find peaks in 2D image."""
    neighborhood_size = MIN_DISTANCE_BETWEEN_TEETH
    local_max = ndi.maximum_filter(img, size=neighborhood_size)
    peaks = (img == local_max) & (img > img.mean())
    return peaks

def segment_individual_teeth(binary_mask):
    """Segment individual teeth with progress tracking."""
    progress.update_progress("Individual Segmentation", 0)
    
    distance = ndi.distance_transform_edt(binary_mask)
    progress.update_progress("Individual Segmentation", 33)
    
    peaks = find_peaks_2d(distance)
    markers = label(peaks)
    progress.update_progress("Individual Segmentation", 66)
    
    labels = watershed(-distance, markers, mask=binary_mask)
    labels = remove_small_objects(labels, min_size=MIN_TOOTH_AREA)
    
    progress.update_progress("Individual Segmentation", 100)
    return labels

def segment_teeth_advanced(image_path):
    """Enhanced teeth segmentation function with individual tooth saving."""
    try:
        progress.current_image = os.path.basename(image_path)
        progress.log(f"Starting segmentation for {progress.current_image}")
        
        # Read image
        img = mpimg.imread(image_path)
        if len(img.shape) == 3:
            img = rgb2gray(img)
        img = img_as_ubyte(img)
        
        # Enhancement
        progress.log("Starting image enhancement")
        enhanced_img = enhance_image(img)
        
        # Boundary detection
        progress.log("Detecting teeth boundaries")
        edges = detect_teeth_boundaries(enhanced_img)
        
        # Segmentation
        progress.log("Performing initial segmentation")
        thresh = threshold_otsu(enhanced_img)
        binary = enhanced_img > thresh
        binary = binary & ~edges
        
        # Morphological operations
        progress.log("Applying morphological operations")
        binary = closing(binary, disk(3))
        binary = opening(binary, disk(2))
        binary = remove_small_objects(binary, min_size=MIN_TOOTH_AREA)
        
        # Individual teeth segmentation
        progress.log("Segmenting individual teeth")
        labels = segment_individual_teeth(binary)
        
        # Process and save individual teeth
        progress.log("Processing individual teeth")
        segmented_teeth = np.zeros_like(img, dtype=np.uint8)
        tooth_count = 0
        saved_paths = []
        
        for region in regionprops(labels):
            if (MIN_TOOTH_AREA < region.area < MAX_TOOTH_AREA and
                region.eccentricity < ECCENTRICITY_THRESHOLD and
                region.solidity > SOLIDITY_THRESHOLD):
                
                tooth_count += 1
                tooth_mask = labels == region.label
                segmented_teeth[tooth_mask] = 255
                
                # Determine quadrant based on centroid
                quadrant = determine_quadrant_and_position(region.centroid, img.shape)
                
                # Save individual tooth
                saved_path = save_individual_tooth(
                    img,
                    tooth_mask,
                    os.path.basename(image_path),
                    tooth_count,
                    quadrant,
                    LOCAL_SEGMENTED_DIR
                )
                saved_paths.append(saved_path)
        
        progress.log(f"Segmentation complete. Saved {tooth_count} individual teeth")
        return segmented_teeth, enhanced_img, saved_paths
        
    except Exception as e:
        progress.log(f"Error in segmentation: {str(e)}")
        return None, None, []

def display_enhanced_images(image_paths, segmented_images, enhanced_images, images_per_row=2):
    """Display images with progress tracking."""
    progress.log("Starting image display")
    
    num_images = len(image_paths)
    if num_images == 0:
        progress.log("No images to display")
        return

    num_rows = (num_images + images_per_row - 1) // images_per_row
    fig, axes = plt.subplots(num_rows, images_per_row * 3, figsize=(18, 4 * num_rows))
    
    if num_rows == 1:
        axes = [axes]
    axes = [ax for sublist in axes for ax in sublist]

    for i, (image_path, segmented_image, enhanced_image) in enumerate(zip(
            tqdm(image_paths, desc="Displaying images"), segmented_images, enhanced_images)):
        try:
            # Original Image
            img = mpimg.imread(image_path)
            if len(img.shape) == 3:
                img = rgb2gray(img)
            axes[i * 3].imshow(img, cmap='gray')
            axes[i * 3].axis('off')
            axes[i * 3].set_title(f"Original: {os.path.basename(image_path)}")

            # Enhanced Image
            if enhanced_image is not None:
                axes[i * 3 + 1].imshow(enhanced_image, cmap='gray')
                axes[i * 3 + 1].axis('off')
                axes[i * 3 + 1].set_title('Enhanced')

            # Segmented Image
            if segmented_image is not None:
                axes[i * 3 + 2].imshow(segmented_image, cmap='gray')
                axes[i * 3 + 2].axis('off')
                axes[i * 3 + 2].set_title('Segmented Teeth')

        except Exception as e:
            progress.log(f"Error displaying image {image_path}: {e}")

    for i in range(num_images * 3, len(axes)):
        axes[i].axis('off')

    plt.tight_layout()
    progress.log("Display complete")
    plt.show()

def list_images_in_s3(bucket, prefix):
    """List all images in S3 bucket with given prefix."""
    try:
        s3_client = boto3.client('s3')
        paginator = s3_client.get_paginator('list_objects_v2')
        pages = paginator.paginate(Bucket=bucket, Prefix=prefix)
        
        image_keys = []
        for page in pages:
            if 'Contents' in page:
                for obj in page['Contents']:
                    if obj['Key'].lower().endswith(('.png', '.jpg', '.jpeg')):
                        image_keys.append(obj['Key'])
        return image_keys
    except Exception as e:
        progress.log(f"Error listing S3 images: {str(e)}")
        return []

def download_image_from_s3(bucket, key, local_path):
    """Download image from S3 to local path."""
    try:
        s3_client = boto3.client('s3')
        s3_client.download_file(bucket, key, local_path)
        return True
    except Exception as e:
        progress.log(f"Error downloading {key}: {str(e)}")
        return False

def main():
    """Main function with progress tracking."""
    progress.log("Starting dental image segmentation pipeline")
    
    # List images
    progress.log("Listing images from S3")
    image_keys = list_images_in_s3(S3_BUCKET, S3_ORIGINAL_IMAGES_PATH)

    if not image_keys:
        progress.log("No images found in the specified S3 path")
        return

    progress.total_images = len(image_keys)
    progress.log(f"Found {progress.total_images} images to process")

    # Download images
    local_image_paths = []
    for key in tqdm(image_keys, desc="Downloading images"):
        local_image_path = os.path.join(LOCAL_IMAGE_DIR, os.path.basename(key))
        if download_image_from_s3(S3_BUCKET, key, local_image_path):
            local_image_paths.append(local_image_path)
            progress.log(f"Downloaded: {os.path.basename(key)}")

    # Process images
    segmented_images = []
    enhanced_images = []
    all_saved_paths = []
    
    for image_path in tqdm(local_image_paths, desc="Processing images"):
        progress.processed_images += 1
        progress.log(f"Processing image {progress.processed_images}/{progress.total_images}")
        
        segmented_teeth, enhanced_img, saved_paths = segment_teeth_advanced(image_path)
        segmented_images.append(segmented_teeth)
        enhanced_images.append(enhanced_img)
        all_saved_paths.extend(saved_paths)

    # Display results
    if local_image_paths:
        display_enhanced_images(local_image_paths, segmented_images, enhanced_images)

    # Save logs and report
    total_time = progress.get_elapsed_time()
    progress.log(f"Pipeline complete. Total processing time: {total_time:.2f} seconds")
    progress.log(f"Average time per image: {total_time/progress.total_images:.2f} seconds")
    progress.log(f"Total individual teeth saved: {len(all_saved_paths)}")
    progress.save_logs()

if __name__ == "__main__":
    main()

In [ ]:
import boto3
import io
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

# AWS S3 Configuration
S3_BUCKET = 'sagemaker-studio-010526272250-hsf94lgtf6i'  # Replace with your bucket name
S3_SEGMENTED_PATH = 'segmented_teeth_output/'  # Path to segmented teeth in S3

# Laptop Local Drive Configuration - Replace with your desired path
LAPTOP_DOWNLOAD_DIR = os.path.join(os.path.expanduser("~"), "Downloads", "segmented_teeth")  # Example: Downloads/segmented_teeth
os.makedirs(LAPTOP_DOWNLOAD_DIR, exist_ok=True)


# Initialize S3 client
s3 = boto3.client('s3')


def list_images_in_s3(bucket, prefix):
    """Lists all image files in the specified S3 bucket and prefix."""
    image_keys = []
    try:
        response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
        if 'Contents' in response:
            for obj in response['Contents']:
                if obj['Key'].lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_keys.append(obj['Key'])
        else:
            print(f"No objects found in s3://{bucket}/{prefix}")
    except Exception as e:
        print(f"Error listing objects in S3: {e}")
    return image_keys


def download_image_from_s3(bucket, key, local_path):
    """Downloads an image from S3 to a local file."""
    try:
        s3.download_file(bucket, key, local_path)
        print(f"Downloaded s3://{bucket}/{key} to {local_path}")
        return True
    except Exception as e:
        print(f"Error downloading {key} from S3: {e}")
        return False

def display_images(image_paths, images_per_row=5):
    """Displays images in a grid using Matplotlib."""
    num_images = len(image_paths)
    if num_images == 0:
        print("No images to display.")
        return

    num_rows = (num_images + images_per_row - 1) // images_per_row  # Calculate number of rows
    fig, axes = plt.subplots(num_rows, images_per_row, figsize=(15, 3 * num_rows)) #Adjust fig size as needed

    # Flatten axes array for easier indexing
    if num_rows == 1:
        axes = [axes]  # Handle the case of only one row
    axes = [ax for sublist in axes for ax in sublist]

    for i, image_path in enumerate(image_paths):
        try:
            img = mpimg.imread(image_path)
            axes[i].imshow(img)
            axes[i].axis('off')  # Hide axes
            axes[i].set_title(os.path.basename(image_path)) # add image name to title
        except FileNotFoundError:
            print(f"File not found: {image_path}")
        except Exception as e:
            print(f"Error displaying image {image_path}: {e}")

    # Hide any unused subplots
    for i in range(num_images, len(axes)):
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()


def main():
    """Lists images in S3, downloads them, and displays them."""
    image_keys = list_images_in_s3(S3_BUCKET, S3_SEGMENTED_PATH)

    if not image_keys:
        print("No images found in the specified S3 path.")
        return

    # Download images to a local directory
    local_image_paths = []
    for key in image_keys:
        local_image_path = os.path.join(LAPTOP_DOWNLOAD_DIR, os.path.basename(key)) # Use the new directory
        if download_image_from_s3(S3_BUCKET, key, local_image_path):
            local_image_paths.append(local_image_path)

    # Display the downloaded images
    if local_image_paths:
        display_images(local_image_paths)

    # Cleanup local directory (optional):
    # Warning: This will delete the downloaded images.
    # Uncomment if you want to delete the images after viewing.
    # import shutil
    # shutil.rmtree(LAPTOP_DOWNLOAD_DIR) #CRUCIAL: This line must be commented out!


if __name__ == "__main__":
    main()

#Code Version6

In [ ]:
import boto3
import io
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import numpy as np
from skimage.feature import greycomatrix, graycoprops
from skimage import img_as_ubyte, img_as_float
from skimage.filters import threshold_otsu, rank, gaussian, sobel
from skimage.measure import label, regionprops
from skimage.morphology import closing, square, remove_small_objects, disk, opening, erosion, dilation
from skimage.color import rgb2gray
from skimage.segmentation import watershed, clear_border
from skimage.exposure import equalize_adapthist
from scipy import ndimage as ndi
from scipy.signal import find_peaks
from tqdm import tqdm
import time
from datetime import datetime
import pandas as pd

# AWS S3 Configuration
S3_BUCKET = 'sagemaker-studio-010526272250-hsf94lgtf6i'
S3_ORIGINAL_IMAGES_PATH = 'Dentex_Challenge_Project_Image_Dataset_ultrareduced/'
S3_SEGMENTED_PATH = 'segmented_teeth_output/'
S3_CSV_PATH = 'csv_output/' # New variable for where to store the CSV
LOCAL_IMAGE_DIR = 'downloaded_images'
LOCAL_SEGMENTED_DIR = 'segmented_teeth'
os.makedirs(LOCAL_IMAGE_DIR, exist_ok=True)
os.makedirs(LOCAL_SEGMENTED_DIR, exist_ok=True)

# Enhanced constants
MIN_TOOTH_AREA = 300
MAX_TOOTH_AREA = 15000
ECCENTRICITY_THRESHOLD = 0.85
SOLIDITY_THRESHOLD = 0.45
SMOOTHING_RADIUS = 10
CLAHE_KERNEL = 8
WATERSHED_FOOTPRINT = 3
MIN_DISTANCE_BETWEEN_TEETH = 20

# Dental quadrant mapping
QUADRANT_MAPPING = {
    'upper_right': 1,
    'upper_left': 2,
    'lower_left': 3,
    'lower_right': 4
}

class ProgressTracker:
    def __init__(self):
        self.start_time = time.time()
        self.logs = []
        self.current_image = None
        self.total_images = 0
        self.processed_images = 0
        
    def log(self, message, stage=None):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_entry = f"[{timestamp}] {message}"
        print(log_entry)
        self.logs.append(log_entry)
        
    def update_progress(self, stage, progress):
        self.log(f"Image {self.current_image}: {stage} - {progress}%")
        
    def save_logs(self, filename="segmentation_logs.txt"):
        with open(filename, 'w') as f:
            f.write("\n".join(self.logs))
            
    def get_elapsed_time(self):
        return time.time() - self.start_time

# Initialize global progress tracker
progress = ProgressTracker()

def determine_quadrant_and_position(centroid, image_shape):
    """Determine the dental quadrant and relative position of a tooth."""
    height, width = image_shape
    y, x = centroid
    
    # Determine if upper or lower
    is_upper = y < height / 2
    
    # Determine if left or right
    is_right = x < width / 2
    
    if is_upper and is_right:
        return 1  # Upper right
    elif is_upper and not is_right:
        return 2  # Upper left
    elif not is_upper and not is_right:
        return 3  # Lower left
    else:
        return 4  # Lower right

def save_individual_tooth(image, mask, base_filename, tooth_number, quadrant, output_dir):
    """Save individual tooth image with appropriate naming convention."""
    # Extract the first part of the filename (before underscore)
    base_name = base_filename.split('_')[0]
    
    # Create output filename
    output_filename = f"{base_name}_Q{quadrant}_T{tooth_number}.png"
    output_path = os.path.join(output_dir, output_filename)
    
    # Apply mask to original image
    tooth_image = image.copy()
    tooth_image[~mask] = 0
    
    # Convert to PIL Image and save
    tooth_image_pil = Image.fromarray(img_as_ubyte(tooth_image))
    tooth_image_pil.save(output_path)
    
    return output_path, output_filename

def enhance_image(img):
    """Apply advanced image enhancement techniques with progress tracking."""
    progress.update_progress("Enhancement", 0)
    
    img_float = img_as_float(img)
    progress.update_progress("Enhancement", 25)
    
    img_clahe = equalize_adapthist(img_float, kernel_size=CLAHE_KERNEL)
    progress.update_progress("Enhancement", 50)
    
    img_smooth = gaussian(img_clahe, sigma=1, preserve_range=True)
    progress.update_progress("Enhancement", 75)
    
    result = img_as_ubyte(img_smooth)
    progress.update_progress("Enhancement", 100)
    
    return result

def detect_teeth_boundaries(img):
    """Detect teeth boundaries with progress tracking."""
    progress.update_progress("Boundary Detection", 0)
    
    gradient = sobel(img)
    progress.update_progress("Boundary Detection", 33)
    
    gradient = gaussian(gradient, sigma=1)
    progress.update_progress("Boundary Detection", 66)
    
    grad_thresh = threshold_otsu(gradient)
    edges = gradient > grad_thresh
    edges = closing(edges, disk(2))
    edges = remove_small_objects(edges, min_size=50)
    
    progress.update_progress("Boundary Detection", 100)
    return edges

def find_peaks_2d(img):
    """Find peaks in 2D image."""
    neighborhood_size = MIN_DISTANCE_BETWEEN_TEETH
    local_max = ndi.maximum_filter(img, size=neighborhood_size)
    peaks = (img == local_max) & (img > img.mean())
    return peaks

def segment_individual_teeth(binary_mask):
    """Segment individual teeth with progress tracking."""
    progress.update_progress("Individual Segmentation", 0)
    
    distance = ndi.distance_transform_edt(binary_mask)
    progress.update_progress("Individual Segmentation", 33)
    
    peaks = find_peaks_2d(distance)
    markers = label(peaks)
    progress.update_progress("Individual Segmentation", 66)
    
    labels = watershed(-distance, markers, mask=binary_mask)
    labels = remove_small_objects(labels, min_size=MIN_TOOTH_AREA)
    
    progress.update_progress("Individual Segmentation", 100)
    return labels

def segment_teeth_advanced(image_path):
    """Enhanced teeth segmentation function with individual tooth saving."""
    try:
        progress.current_image = os.path.basename(image_path)
        progress.log(f"Starting segmentation for {progress.current_image}")
        
        # Read image
        img = mpimg.imread(image_path)
        if len(img.shape) == 3:
            img = rgb2gray(img)
        img = img_as_ubyte(img)
        
        # Enhancement
        progress.log("Starting image enhancement")
        enhanced_img = enhance_image(img)
        
        # Boundary detection
        progress.log("Detecting teeth boundaries")
        edges = detect_teeth_boundaries(enhanced_img)
        
        # Segmentation
        progress.log("Performing initial segmentation")
        thresh = threshold_otsu(enhanced_img)
        binary = enhanced_img > thresh
        binary = binary & ~edges
        
        # Morphological operations
        progress.log("Applying morphological operations")
        binary = closing(binary, disk(3))
        binary = opening(binary, disk(2))
        binary = remove_small_objects(binary, min_size=MIN_TOOTH_AREA)
        
        # Individual teeth segmentation
        progress.log("Segmenting individual teeth")
        labels = segment_individual_teeth(binary)
        
        # Process and save individual teeth
        progress.log("Processing individual teeth")
        segmented_teeth = np.zeros_like(img, dtype=np.uint8)
        tooth_count = 0
        saved_data = []
        
        for region in regionprops(labels):
            if (MIN_TOOTH_AREA < region.area < MAX_TOOTH_AREA and
                region.eccentricity < ECCENTRICITY_THRESHOLD and
                region.solidity > SOLIDITY_THRESHOLD):
                
                tooth_count += 1
                tooth_mask = labels == region.label
                segmented_teeth[tooth_mask] = 255
                
                # Determine quadrant based on centroid
                quadrant = determine_quadrant_and_position(region.centroid, img.shape)
                
                # Save individual tooth
                saved_path, output_filename = save_individual_tooth(
                    img,
                    tooth_mask,
                    os.path.basename(image_path),
                    tooth_count,
                    quadrant,
                    LOCAL_SEGMENTED_DIR
                )
                
                # Store data for CSV
                saved_data.append({
                    'image_path': image_path,
                    'tooth_filename': output_filename,
                    'quadrant': quadrant,
                    'tooth_number': tooth_count
                })

        progress.log(f"Segmentation complete. Saved {tooth_count} individual teeth")
        return segmented_teeth, enhanced_img, saved_data
        
    except Exception as e:
        progress.log(f"Error in segmentation: {str(e)}")
        return None, None, []

def display_enhanced_images(image_paths, segmented_images, enhanced_images, images_per_row=2):
    """Display images with progress tracking."""
    progress.log("Starting image display")
    
    num_images = len(image_paths)
    if num_images == 0:
        progress.log("No images to display")
        return

    num_rows = (num_images + images_per_row - 1) // images_per_row
    fig, axes = plt.subplots(num_rows, images_per_row * 3, figsize=(18, 4 * num_rows))
    
    if num_rows == 1:
        axes = [axes]
    axes = [ax for sublist in axes for ax in sublist]

    for i, (image_path, segmented_image, enhanced_image) in enumerate(zip(
            tqdm(image_paths, desc="Displaying images"), segmented_images, enhanced_images)):
        try:
            # Original Image
            img = mpimg.imread(image_path)
            if len(img.shape) == 3:
                img = rgb2gray(img)
            axes[i * 3].imshow(img, cmap='gray')
            axes[i * 3].axis('off')
            axes[i * 3].set_title(f"Original: {os.path.basename(image_path)}")

            # Enhanced Image
            if enhanced_image is not None:
                axes[i * 3 + 1].imshow(enhanced_image, cmap='gray')
                axes[i * 3 + 1].axis('off')
                axes[i * 3 + 1].set_title('Enhanced')

            # Segmented Image
            if segmented_image is not None:
                axes[i * 3 + 2].imshow(segmented_image, cmap='gray')
                axes[i * 3 + 2].axis('off')
                axes[i * 3 + 2].set_title('Segmented Teeth')

        except Exception as e:
            progress.log(f"Error displaying image {image_path}: {e}")

    for i in range(num_images * 3, len(axes)):
        axes[i].axis('off')

    plt.tight_layout()
    progress.log("Display complete")
    plt.show()

def list_images_in_s3(bucket, prefix):
    """List all images in S3 bucket with given prefix."""
    try:
        s3_client = boto3.client('s3')
        paginator = s3_client.get_paginator('list_objects_v2')
        pages = paginator.paginate(Bucket=bucket, Prefix=prefix)
        
        image_keys = []
        for page in pages:
            if 'Contents' in page:
                for obj in page['Contents']:
                    if obj['Key'].lower().endswith(('.png', '.jpg', '.jpeg')):
                        image_keys.append(obj['Key'])
        return image_keys
    except Exception as e:
        progress.log(f"Error listing S3 images: {str(e)}")
        return []

def download_image_from_s3(bucket, key, local_path):
    """Download image from S3 to local path."""
    try:
        s3_client = boto3.client('s3')
        s3_client.download_file(bucket, key, local_path)
        return True
    except Exception as e:
        progress.log(f"Error downloading {key}: {str(e)}")
        return False

def upload_file_to_s3(bucket, local_file, s3_path):
    """Upload a file to an S3 bucket."""
    try:
        s3_client = boto3.client('s3')
        s3_client.upload_file(local_file, bucket, s3_path)
        return True
    except Exception as e:
        progress.log(f"Error uploading {local_file} to {s3_path}: {str(e)}")
        return False

def main():
    """Main function with progress tracking and CSV export to S3."""
    progress.log("Starting dental image segmentation pipeline")
    
    # List images
    progress.log("Listing images from S3")
    image_keys = list_images_in_s3(S3_BUCKET, S3_ORIGINAL_IMAGES_PATH)

    if not image_keys:
        progress.log("No images found in the specified S3 path")
        return

    progress.total_images = len(image_keys)
    progress.log(f"Found {progress.total_images} images to process")

    # Download images
    local_image_paths = []
    for key in tqdm(image_keys, desc="Downloading images"):
        local_image_path = os.path.join(LOCAL_IMAGE_DIR, os.path.basename(key))
        if download_image_from_s3(S3_BUCKET, key, local_image_path):
            local_image_paths.append(local_image_path)
            progress.log(f"Downloaded: {os.path.basename(key)}")

    # Process images
    segmented_images = []
    enhanced_images = []
    all_tooth_data = []  # List to store data for the CSV

    for image_path in tqdm(local_image_paths, desc="Processing images"):
        progress.processed_images += 1
        progress.log(f"Processing image {progress.processed_images}/{progress.total_images}")
        
        segmented_teeth, enhanced_img, tooth_data = segment_teeth_advanced(image_path)
        segmented_images.append(segmented_teeth)
        enhanced_images.append(enhanced_img)
        all_tooth_data.extend(tooth_data)

    # Display results
    if local_image_paths:
        display_enhanced_images(local_image_paths, segmented_images, enhanced_images)

    # Create and save CSV file
    progress.log("Creating CSV file")
    df = pd.DataFrame(all_tooth_data)
    csv_filename = "segmented_teeth_data.csv"
    local_csv_path = csv_filename  # Save to the current directory temporarily
    df.to_csv(local_csv_path, index=False)

    # Upload the CSV to S3
    s3_csv_path = os.path.join(S3_CSV_PATH, csv_filename)
    if upload_file_to_s3(S3_BUCKET, local_csv_path, s3_csv_path):
        progress.log(f"CSV file uploaded to s3://{S3_BUCKET}/{s3_csv_path}")
        os.remove(local_csv_path) # Clean up the local file
    else:
        progress.log("Failed to upload CSV to S3.")

    # Save logs and report
    total_time = progress.get_elapsed_time()
    progress.log(f"Pipeline complete. Total processing time: {total_time:.2f} seconds")
    progress.log(f"Average time per image: {total_time/progress.total_images:.2f} seconds")
    progress.log(f"Total individual teeth saved: {len(all_tooth_data)}")
    progress.save_logs()

if __name__ == "__main__":
    main()

In [ ]:
import boto3
from pathlib import Path
import os

def setup_s3_client():
    """Create and return an S3 client"""
    return boto3.client('s3')

def list_s3_objects(bucket_name, prefix):
    """List all objects in an S3 bucket with given prefix"""
    s3_client = setup_s3_client()
    try:
        response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
        if 'Contents' in response:
            return [obj['Key'] for obj in response['Contents']]
        return []
    except Exception as e:
        print(f"Failed to list objects: {e}")
        return []

def download_from_s3(bucket_name, s3_path, local_base_path):
    """Download files from S3 directory"""
    s3_client = setup_s3_client()
    
    # List all objects in the S3 directory
    objects = list_s3_objects(bucket_name, s3_path)
    
    if not objects:
        print(f"No objects found in {bucket_name}/{s3_path}")
        return
    
    for obj_key in objects:
        # Create local path maintaining the same structure
        relative_path = obj_key.replace(s3_path, '', 1).lstrip('/')
        local_path = os.path.join(local_base_path, relative_path)
        
        # Create directory if it doesn't exist
        Path(local_path).parent.mkdir(parents=True, exist_ok=True)
        
        try:
            # Skip if the object key ends with a forward slash (directory)
            if obj_key.endswith('/'):
                continue
                
            s3_client.download_file(bucket_name, obj_key, str(local_path))
            print(f"Successfully downloaded {bucket_name}/{obj_key} to {local_path}")
        except Exception as e:
            print(f"Download failed for {obj_key}: {e}")

if __name__ == "__main__":
    # Replace with your values
    BUCKET_NAME = "sagemaker-studio-010526272250-hsf94lgtf6i"
    LOCAL_PATH = r"C:\Users\dell\Desktop\Arpita\Arpita M.Tech Assignments and curriculam\4th Sem"
    S3_PATH = "segmented_teeth_output/"
    
    # Download all files from the S3 directory
    download_from_s3(BUCKET_NAME, S3_PATH, LOCAL_PATH)

#Code Version7

In [1]:
import boto3
import io
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import numpy as np
from skimage.feature import greycomatrix, graycoprops
from skimage import img_as_ubyte, img_as_float
from skimage.filters import threshold_otsu, rank, gaussian, sobel
from skimage.measure import label, regionprops
from skimage.morphology import closing, square, remove_small_objects, disk, opening, erosion, dilation
from skimage.color import rgb2gray
from skimage.segmentation import watershed, clear_border
from skimage.exposure import equalize_adapthist
from scipy import ndimage as ndi
from scipy.signal import find_peaks
from tqdm import tqdm
import time
from datetime import datetime
import pandas as pd
import gc  # Import garbage collection

# AWS S3 Configuration
S3_BUCKET = 'sagemaker-studio-010526272250-hsf94lgtf6i'
S3_ORIGINAL_IMAGES_PATH = 'Dentex_Challenge_Project_Image_Dataset_ultrareduced/'
S3_SEGMENTED_PATH = 'segmented_teeth_output/'
S3_CSV_PATH = 'csv_output/'
LOCAL_IMAGE_DIR = 'downloaded_images'
LOCAL_SEGMENTED_DIR = 'segmented_teeth'
os.makedirs(LOCAL_IMAGE_DIR, exist_ok=True)
os.makedirs(LOCAL_SEGMENTED_DIR, exist_ok=True)

# Enhanced constants
MIN_TOOTH_AREA = 300
MAX_TOOTH_AREA = 15000
ECCENTRICITY_THRESHOLD = 0.85
SOLIDITY_THRESHOLD = 0.45
SMOOTHING_RADIUS = 10
CLAHE_KERNEL = 8
WATERSHED_FOOTPRINT = 3
MIN_DISTANCE_BETWEEN_TEETH = 20
RESIZE_FACTOR = 0.5  # Added resize factor to reduce memory

class ProgressTracker:
    def __init__(self):
        self.start_time = time.time()
        self.logs = []
        self.current_image = None
        self.total_images = 0
        self.processed_images = 0

    def log(self, message, stage=None):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_entry = f"[{timestamp}] {message}"
        print(log_entry)
        self.logs.append(log_entry)

    def update_progress(self, stage, progress):
        self.log(f"Image {self.current_image}: {stage} - {progress}%")

    def save_logs(self, filename="segmentation_logs.txt"):
        with open(filename, 'w') as f:
            f.write("\n".join(self.logs))

    def get_elapsed_time(self):
        return time.time() - self.start_time

# Initialize global progress tracker
progress = ProgressTracker()

class ToothSegment:
    def __init__(self, image_data, mask, quadrant, tooth_number, filename):
        self.image_data = image_data
        self.mask = mask
        self.quadrant = quadrant
        self.tooth_number = tooth_number
        self.filename = filename

def enhance_image(img):
    """Apply advanced image enhancement techniques with progress tracking."""
    progress.update_progress("Enhancement", 0)

    img_float = img_as_float(img)
    progress.update_progress("Enhancement", 25)

    img_clahe = equalize_adapthist(img_float, kernel_size=CLAHE_KERNEL)
    progress.update_progress("Enhancement", 50)

    img_smooth = gaussian(img_clahe, sigma=1, preserve_range=True)
    progress.update_progress("Enhancement", 75)

    result = img_as_ubyte(img_smooth)
    progress.update_progress("Enhancement", 100)

    return result

def detect_teeth_boundaries(img):
    """Detect teeth boundaries with progress tracking."""
    progress.update_progress("Boundary Detection", 0)

    gradient = sobel(img)
    progress.update_progress("Boundary Detection", 33)

    gradient = gaussian(gradient, sigma=1)
    progress.update_progress("Boundary Detection", 66)

    grad_thresh = threshold_otsu(gradient)
    edges = gradient > grad_thresh
    edges = closing(edges, disk(2))
    edges = remove_small_objects(edges, min_size=50)

    progress.update_progress("Boundary Detection", 100)
    return edges

def determine_quadrant_and_position(centroid, image_shape):
    """Determine the dental quadrant and relative position of a tooth."""
    height, width = image_shape
    y, x = centroid

    # Determine if upper or lower
    is_upper = y < height / 2

    # Determine if left or right
    is_right = x < width / 2

    if is_upper and is_right:
        return 1  # Upper right
    elif is_upper and not is_right:
        return 2  # Upper left
    elif not is_upper and not is_right:
        return 3  # Lower left
    else:
        return 4  # Lower right

def save_individual_tooth(image, mask, base_filename, tooth_number, quadrant, output_dir):
    """Save individual tooth image and return image data."""
    base_name = base_filename.split('_')[0]
    output_filename = f"{base_name}_Q{quadrant}_T{tooth_number}.png"
    output_path = os.path.join(output_dir, output_filename)

    # Apply mask to original image
    tooth_image = image.copy()
    tooth_image[~mask] = 0

    # Convert to PIL Image and save
    tooth_image_pil = Image.fromarray(img_as_ubyte(tooth_image))
    tooth_image_pil.save(output_path)
    tooth_image_pil.close()  # close image to free memory
    del tooth_image_pil #remove the object
    # Create ToothSegment object
    tooth_segment = ToothSegment(
        image_data=tooth_image,
        mask=mask,
        quadrant=quadrant,
        tooth_number=tooth_number,
        filename=output_filename
    )
    del tooth_image  # explicitly delete to free memory
    gc.collect()  # Force garbage collection
    return output_path, output_filename, tooth_segment

def segment_teeth_advanced(image_path):
    """Enhanced teeth segmentation function with quadrant-based segmentation."""
    try:
        progress.current_image = os.path.basename(image_path)
        progress.log(f"Starting segmentation for {progress.current_image}")

        # Read image
        img = mpimg.imread(image_path)
        if len(img.shape) == 3:
            img = rgb2gray(img)
        img = img_as_ubyte(img)

        # Resize image to reduce memory usage
        img = np.array(Image.fromarray(img).resize((int(img.shape[1] * RESIZE_FACTOR), int(img.shape[0] * RESIZE_FACTOR))))
        # Enhancement
        enhanced_img = enhance_image(img)

        # Boundary detection
        edges = detect_teeth_boundaries(enhanced_img)

        # Initial segmentation
        thresh = threshold_otsu(enhanced_img)
        binary = enhanced_img > thresh
        binary = binary & ~edges

        # Morphological operations
        binary = closing(binary, disk(3))
        binary = opening(binary, disk(2))
        binary = remove_small_objects(binary, min_size=MIN_TOOTH_AREA)

        # Label regions
        labels = label(binary)

        # Initialize quadrant-based storage
        quadrant_teeth = {1: [], 2: [], 3: [], 4: []}
        saved_data = []

        tooth_count = 0
        segmented_teeth = np.zeros_like(img, dtype=np.uint8)

        for region in regionprops(labels):
            if (MIN_TOOTH_AREA < region.area < MAX_TOOTH_AREA and
                region.eccentricity < ECCENTRICITY_THRESHOLD and
                region.solidity > SOLIDITY_THRESHOLD):

                tooth_count += 1
                tooth_mask = labels == region.label
                segmented_teeth[tooth_mask] = 255

                # Determine quadrant
                quadrant = determine_quadrant_and_position(region.centroid, img.shape)

                # Save individual tooth and get segment data
                saved_path, output_filename, tooth_segment = save_individual_tooth(
                    img,
                    tooth_mask,
                    os.path.basename(image_path),
                    tooth_count,
                    quadrant,
                    LOCAL_SEGMENTED_DIR
                )

                # Store segment in appropriate quadrant
                quadrant_teeth[quadrant].append(tooth_segment)

                # Store data for CSV
                tooth_data = {
                    'image_path': image_path,
                    'tooth_filename': output_filename,
                    'quadrant': quadrant,
                    'tooth_number': tooth_count,
                    'upper_right_segment': None,
                    'upper_left_segment': None,
                    'lower_left_segment': None,
                    'lower_right_segment': None
                }

                # Set the segment filename in the appropriate column
                quadrant_column_mapping = {
                    1: 'upper_right_segment',
                    2: 'upper_left_segment',
                    3: 'lower_left_segment',
                    4: 'lower_right_segment'
                }
                tooth_data[quadrant_column_mapping[quadrant]] = output_filename

                saved_data.append(tooth_data)
        # Explicitly delete large variables after use
        del labels, binary, edges, enhanced_img, img
        gc.collect()  # Force garbage collection
        progress.log(f"Segmentation complete. Saved {tooth_count} individual teeth")
        return segmented_teeth, saved_data, quadrant_teeth

    except Exception as e:
        progress.log(f"Error in segmentation: {str(e)}")
        return None, [], {}

def list_images_in_s3(bucket, prefix):
    """List all images in S3 bucket with given prefix."""
    try:
        s3_client = boto3.client('s3')
        paginator = s3_client.get_paginator('list_objects_v2')
        pages = paginator.paginate(Bucket=bucket, Prefix=prefix)

        image_keys = []
        for page in pages:
            if 'Contents' in page:
                for obj in page['Contents']:
                    if obj['Key'].lower().endswith(('.png', '.jpg', '.jpeg')):
                        image_keys.append(obj['Key'])
        return image_keys
    except Exception as e:
        progress.log(f"Error listing S3 images: {str(e)}")
        return []

def download_image_from_s3(bucket, key, local_path):
    """Download image from S3 to local path."""
    try:
        s3_client = boto3.client('s3')
        s3_client.download_file(bucket, key, local_path)
        return True
    except Exception as e:
        progress.log(f"Error downloading {key}: {str(e)}")
        return False

def upload_file_to_s3(bucket, local_file, s3_path):
    """Upload a file to an S3 bucket."""
    try:
        s3_client = boto3.client('s3')
        s3_client.upload_file(local_file, bucket, s3_path)
        return True
    except Exception as e:
        progress.log(f"Error uploading {local_file} to {s3_path}: {str(e)}")
        return False


def process_image_and_append_data(image_path, all_tooth_data, all_quadrant_data):
    """Process a single image and append its data to the cumulative lists."""
    segmented_teeth, tooth_data, quadrant_teeth = segment_teeth_advanced(image_path)

    # Merge quadrant data
    for quadrant in range(1, 5):
        all_quadrant_data[quadrant].extend(quadrant_teeth.get(quadrant, []))

    if tooth_data:  # Only extend if there's valid data
        all_tooth_data.extend(tooth_data)

    # Explicitly delete large variables after use
    del segmented_teeth, quadrant_teeth
    gc.collect()  # Run garbage collector


def write_data_to_csv(all_tooth_data, S3_BUCKET, S3_CSV_PATH):
    """Write tooth data to a CSV file and upload it to S3."""
    if not all_tooth_data:
        progress.log("No tooth data was generated, skipping CSV creation")
        return

    progress.log("Creating enhanced CSV file")
    df = pd.DataFrame(all_tooth_data)

    # Add statistics for each quadrant
    df['upper_right_count'] = df.groupby('image_path')['upper_right_segment'].transform('count')
    df['upper_left_count'] = df.groupby('image_path')['upper_left_segment'].transform('count')
    df['lower_left_count'] = df.groupby('image_path')['lower_left_segment'].transform('count')
    df['lower_right_count'] = df.groupby('image_path')['lower_right_segment'].transform('count')

    # Save CSV with enhanced information
    csv_filename = "segmented_teeth_data_enhanced.csv"
    local_csv_path = csv_filename
    df.to_csv(local_csv_path, index=False)

    # Upload to S3
    s3_csv_path = os.path.join(S3_CSV_PATH, csv_filename)
    if upload_file_to_s3(S3_BUCKET, local_csv_path, s3_csv_path):
        progress.log(f"CSV file uploaded to s3://{S3_BUCKET}/{s3_csv_path}")
        os.remove(local_csv_path)  # Clean up the local file
    else:
        progress.log("Failed to upload CSV to S3")

    del df  # Explicitly delete DataFrame
    gc.collect()  # Garbage collection



def main():
    """Enhanced main function with quadrant-based processing."""
    progress.log("Starting dental image segmentation pipeline")

    # List images
    progress.log("Listing images from S3")
    image_keys = list_images_in_s3(S3_BUCKET, S3_ORIGINAL_IMAGES_PATH)

    if not image_keys:
        progress.log("No images found in the specified S3 path")
        return

    progress.total_images = len(image_keys)
    progress.log(f"Found {progress.total_images} images to process")

    # Download images
    local_image_paths = []
    for key in tqdm(image_keys, desc="Downloading images"):
        local_image_path = os.path.join(LOCAL_IMAGE_DIR, os.path.basename(key))
        if download_image_from_s3(S3_BUCKET, key, local_image_path):
            local_image_paths.append(local_image_path)
            progress.log(f"Downloaded: {os.path.basename(key)}")
    # Initialize lists to store data
    all_tooth_data = []
    all_quadrant_data = {1: [], 2: [], 3: [], 4: []}

    for image_path in tqdm(local_image_paths, desc="Processing images"):
        progress.processed_images += 1
        progress.log(f"Processing image {progress.processed_images}/{progress.total_images}")

        process_image_and_append_data(image_path, all_tooth_data, all_quadrant_data)

    write_data_to_csv(all_tooth_data, S3_BUCKET, S3_CSV_PATH)

    # Save logs and report
    total_time = progress.get_elapsed_time()
    progress.log(f"Pipeline complete. Total processing time: {total_time:.2f} seconds")
    progress.log(f"Total individual teeth saved: {len(all_tooth_data)}")
    progress.save_logs()


if __name__ == "__main__":
    main()

[2025-02-13 05:53:47] Starting dental image segmentation pipeline
[2025-02-13 05:53:47] Listing images from S3
[2025-02-13 05:53:47] Found 78 images to process


[2025-02-13 05:53:47] Downloaded: Quadrant-Enumeration-Disease_train124.png


[2025-02-13 05:53:47] Downloaded: Quadrant-Enumeration-Disease_train125.png


[2025-02-13 05:53:47] Downloaded: Quadrant-Enumeration-Disease_train126.png
[2025-02-13 05:53:47] Downloaded: Quadrant-Enumeration-Disease_train127.png


[2025-02-13 05:53:48] Downloaded: Quadrant-Enumeration-Disease_train128.png
[2025-02-13 05:53:48] Downloaded: Quadrant-Enumeration-Disease_train129.png


...

[2025-02-13 05:57:06] Segmentation complete. Saved 19 individual teeth
[2025-02-13 05:57:06] Processing image 77/78
[2025-02-13 05:57:06] Starting segmentation for Quadrant-Enumeration-Disease_train200.png
[2025-02-13 05:57:06] Image Quadrant-Enumeration-Disease_train200.png: Enhancement - 0%
[2025-02-13 05:57:06] Image Quadrant-Enumeration-Disease_train200.png: Enhancement - 25%
[2025-02-13 05:57:07] Image Quadrant-Enumeration-Disease_train200.png: Enhancement - 50%
[2025-02-13 05:57:07] Image Quadrant-Enumeration-Disease_train200.png: Enhancement - 75%
[2025-02-13 05:57:07] Image Quadrant-Enumeration-Disease_train200.png: Enhancement - 100%
[2025-02-13 05:57:07] Image Quadrant-Enumeration-Disease_train200.png: Boundary Detection - 0%
[2025-02-13 05:57:07] Image Quadrant-Enumeration-Disease_train200.png: Boundary Detection - 33%
[2025-02-13 05:57:07] Image Quadrant-Enumeration-Disease_train200.png: Boundary Detection - 66%
[2025-02-13 05:57:07] Image Quadrant-Enumeration-Disease_train

Processing images:  99%|█████████▊| 77/78 [03:09<00:02,  2.49s/it]

[2025-02-13 05:57:09] Segmentation complete. Saved 18 individual teeth
[2025-02-13 05:57:09] Processing image 78/78
[2025-02-13 05:57:09] Starting segmentation for Quadrant-Enumeration-Disease_train201.png
[2025-02-13 05:57:09] Image Quadrant-Enumeration-Disease_train201.png: Enhancement - 0%
[2025-02-13 05:57:09] Image Quadrant-Enumeration-Disease_train201.png: Enhancement - 25%
[2025-02-13 05:57:10] Image Quadrant-Enumeration-Disease_train201.png: Enhancement - 50%
[2025-02-13 05:57:10] Image Quadrant-Enumeration-Disease_train201.png: Enhancement - 75%
[2025-02-13 05:57:10] Image Quadrant-Enumeration-Disease_train201.png: Enhancement - 100%
[2025-02-13 05:57:10] Image Quadrant-Enumeration-Disease_train201.png: Boundary Detection - 0%
[2025-02-13 05:57:10] Image Quadrant-Enumeration-Disease_train201.png: Boundary Detection - 33%
[2025-02-13 05:57:10] Image Quadrant-Enumeration-Disease_train201.png: Boundary Detection - 66%
[2025-02-13 05:57:10] Image Quadrant-Enumeration-Disease_train

Processing images: 100%|██████████| 78/78 [03:12<00:00,  2.47s/it]

[2025-02-13 05:57:12] Segmentation complete. Saved 24 individual teeth
[2025-02-13 05:57:12] Creating enhanced CSV file
[2025-02-13 05:57:12] CSV file uploaded to s3://sagemaker-studio-010526272250-hsf94lgtf6i/csv_output/segmented_teeth_data_enhanced.csv
[2025-02-13 05:57:12] Pipeline complete. Total processing time: 205.73 seconds
[2025-02-13 05:57:12] Total individual teeth saved: 1334


#Tooth Segmentation V1(Contour wise segmentaion)

In [ ]:
import tensorflow as tf
import numpy as np
import boto3
import io
import os
from PIL import Image
import scipy.ndimage as ndimage

# AWS S3 Configuration
s3_client = boto3.client('s3')
input_bucket = 'sagemaker-studio-010526272250-hsf94lgtf6i'
input_prefix = 'Dentex_Challenge_Project_Image_Dataset_reduced'
output_bucket = 'sagemaker-studio-010526272250-hsf94lgtf6i'
output_prefix = 'Dentex_Challenge_Project_Image_Dataset_reduced_segmented_output'
model_bucket = 'sagemaker-studio-010526272250-hsf94lgtf6i'
model_key = 'dental_xray_seg.h5'

def load_model_from_s3():
    """
    Load the model directly from S3
    """
    local_model_path = '/tmp/dental_xray_seg.h5'
    s3_client.download_file(Bucket=model_bucket, Key=model_key, Filename=local_model_path)
    model = tf.keras.models.load_model(local_model_path)
    os.remove(local_model_path)
    return model

# Load the model
model = load_model_from_s3()

def load_image_from_s3(bucket, key):
    """
    Load an image directly from S3 using boto3 and PIL
    """
    response = s3_client.get_object(Bucket=bucket, Key=key)
    image_stream = response['Body']
    image = Image.open(image_stream).convert('L')  # Convert to grayscale
    return np.array(image)

def save_image_to_s3(bucket, key, image):
    """
    Save an image to S3 using PIL
    """
    pil_image = Image.fromarray(image)
    byte_arr = io.BytesIO()
    pil_image.save(byte_arr, format='PNG')
    byte_arr.seek(0)
    s3_client.put_object(Bucket=bucket, Key=key, Body=byte_arr)

def preprocess_image(img):
    """
    Image preprocessing for segmentation
    """
    img = img.astype(np.float32)
    img_resized = tf.image.resize(img[..., np.newaxis], (512, 512), method=tf.image.ResizeMethod.LANCZOS3, antialias=True)
    img_normalized = img_resized / 255.0
    img_input = tf.reshape(img_normalized, (1, 512, 512, 1))
    return img_input

def create_segmentation_mask(prediction, original_shape):
    """
    Create segmentation mask from model prediction
    """
    mask = tf.image.resize(prediction, original_shape[:2], method=tf.image.ResizeMethod.LANCZOS3).numpy()[0, :, :, 0]
    return (mask > 0.5).astype(np.uint8) * 255

def find_contours(mask):
    """
    Find contours using scipy
    """
    labeled, num_features = ndimage.label(mask)
    return [np.column_stack(np.where(labeled == i)) for i in range(1, num_features + 1)]

def extract_teeth(img, contours):
    """
    Extract individual teeth from the image
    """
    return [img[y.min():y.max()+1, x.min():x.max()+1] for cnt in contours for x, y in [cnt.T]]

def draw_contours(img, contours):
    """
    Draw contours on the image
    """
    img_with_contours = np.stack([img] * 3, axis=-1) if len(img.shape) == 2 else img.copy()
    for cnt in contours:
        for point in cnt:
            img_with_contours[point[0], point[1]] = [255, 0, 0]
    return img_with_contours

def main():
    """
    Main processing function
    """
    paginator = s3_client.get_paginator('list_objects_v2')
    for result in paginator.paginate(Bucket=input_bucket, Prefix=input_prefix):
        for obj in result.get('Contents', []):
            if obj['Key'].endswith('/') or not obj['Key'].lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff')):
                continue
            try:
                print(f"Processing image: {obj['Key']}")
                img = load_image_from_s3(input_bucket, obj['Key'])
                img_input = preprocess_image(img)
                prediction = model.predict(img_input)
                mask = create_segmentation_mask(prediction, img.shape)
                contours = find_contours(mask)
                output_with_contours = draw_contours(img, contours)
                output_key = os.path.join(output_prefix, 'contoured', os.path.basename(obj['Key']))
                save_image_to_s3(output_bucket, output_key, output_with_contours)
                for i, tooth in enumerate(extract_teeth(img, contours)):
                    tooth_key = os.path.join(output_prefix, 'extracted_teeth', f"{os.path.splitext(os.path.basename(obj['Key']))[0]}_tooth_{i+1}.png")
                    save_image_to_s3(output_bucket, tooth_key, tooth)
                print(f"Processed and saved outputs for {obj['Key']}")
            except Exception as e:
                print(f"Error processing {obj['Key']}: {str(e)}")

if __name__ == "__main__":
    main()


Processing image: Dentex_Challenge_Project_Image_Dataset_reduced/Quadrant-Enumeration-Disease_train124.png
1/1 [==============================] - 1s 845ms/step
Error processing Dentex_Challenge_Project_Image_Dataset_reduced/Quadrant-Enumeration-Disease_train124.png: tile cannot extend outside image
Processing image: Dentex_Challenge_Project_Image_Dataset_reduced/Quadrant-Enumeration-Disease_train125.png
1/1 [==============================] - 1s 625ms/step
Error processing Dentex_Challenge_Project_Image_Dataset_reduced/Quadrant-Enumeration-Disease_train125.png: tile cannot extend outside image
Processing image: Dentex_Challenge_Project_Image_Dataset_reduced/Quadrant-Enumeration-Disease_train126.png
1/1 [==============================] - 1s 627ms/step
Error processing Dentex_Challenge_Project_Image_Dataset_reduced/Quadrant-Enumeration-Disease_train126.png: tile cannot extend outside image
Processing image: Dentex_Challenge_Project_Image_Dataset_reduced/Quadrant-Enumeration-Disease_train1